# Food Classification with CNN - Building a Restaurant Recommendation System

This assignment focuses on developing a deep learning-based food classification system using Convolutional Neural Networks (CNNs). You will build a model that can recognize different food categories and use it to return the food preferences of a user.

## Learning Objectives
- Implement CNNs for image classification
- Work with real-world food image datasets
- Build a preference-detector system

## Background: AI-Powered Food Preference Discovery

The system's core idea is simple:

1. Users upload 10 photos of dishes they enjoy
2. Your CNN classifies these images into the 91 categories
3. Based on these categories, the system returns the user's taste profile

Your task is to develop the core computer vision component that will power this detection engine.

You are given a training ("train" folder) and a test ("test" folder) dataset which have ~45k and ~22k samples respectively. For each one of the 91 classes there is a subdirectory containing the images of the respective class.

## Assignment Requirements

### Technical Requirements
- Implement your own pytorch CNN architecture for food image classification
- Use only the provided training dataset split for training
- Train the network from scratch ; No pretrained weights can be used
- Report test-accuracy after every epoch
- Report all hyperparameters of final model
- Use a fixed seed and do not use any CUDA-features that break reproducibility
- Use Pytorch 2.6

### Deliverables
1. Jupyter Notebook with CNN implementation, training code etc.
2. README file
3. Report (max 3 pages)

Submit your report, README and all code files as a single zip file named GROUP_[number]_NC2425_PA. The names and IDs of the group components must be mentioned in the README.
Do not include the dataset in your submission.

### Grading

1. Correct CNN implementation, training runs on the uni DSLab computers according to the README.MD instructions without ANY exceptions on the DSLab machines: 3pt
2. Perfect 1:1 reproducibility on DSLab machines: 1pt
3. Very clear github-repo-style README.MD with instructions for running the code: 1pt
4. Report: 1pt
5. Model test performance on test-set: interpolated from 30-80% test-accuracy: 0-3pt
6. Pick 10 random pictures of the test set to simulate a user uploading images and report which categories occur how often in these: 1pt
7. Bonus point: use an LLM (API) to generate short description / profile of preferences of the simulated user

**If there is anything unclear about this assignment please post your question in the Brightspace discussions forum or send an email**


# Loading the datasets
The dataset is already split into a train and test set in the directories "train" and "test". 

In [ ]:
### HYPER-PARAMETER SET-UP ###

# Randomization
random_seed = 42
# Model training
workers = 4
batch_size = 16
num_epochs = 10
learning_rate = 0.001
momentum = 0.5

In [2]:
### IMPORTS AND DEVICE SELECTION ###

import torch
import numpy as np
import torch.nn as nn
from time import time
import torch.nn.functional as F
from datetime import datetime as dt
from torch.utils.data import DataLoader
from torchvision import datasets, transforms


# Use the GPU to speed up the training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
### DATA LOADING ###

# Allow repeatability
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)

# Define the transformer for the data
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])


full_dataset = datasets.ImageFolder(root='./train', transform=transform)
train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [0.9, 0.1])
test_dataset = datasets.ImageFolder(root='./test', transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=workers)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False, num_workers=workers)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False, num_workers=workers)

classes = test_dataset.classes

# CNN Implementation

In [4]:
### NETWORK ARCHITECHTURE ####

class FoodCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, 1, 1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(2, 2)  # 256 -> 128

        self.conv2 = nn.Conv2d(32, 64, 3, 1, 1)
        self.pool2 = nn.MaxPool2d(2, 2)  # 128 -> 64

        self.conv3 = nn.Conv2d(64, 128, 3, 1, 1)
        self.conv4 = nn.Conv2d(128, 256, 2, 2, 0)  # 64 -> 32
        self.bn2 = nn.BatchNorm2d(256)
        self.pool3 = nn.AvgPool2d(2, 2)  # 32 -> 16

        self.conv5 = nn.Conv2d(256, 512, 3, 1, 1)
        self.pool4 = nn.MaxPool2d(2, 2)  # 16 -> 8

        self.flat = nn.Flatten()

        self.fc1 = nn.Linear(512*8*8, 512*8)
        self.drop1 = nn.Dropout()

        self.fc2 = nn.Linear(512*8, 512)
        self.drop2 = nn.Dropout()

        self.fc3 = nn.Linear(512, 91)


    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.pool1(x)

        x = F.relu(self.conv2(x))
        x = self.pool2(x)

        x = F.relu(self.conv3(x))
        x = F.relu(self.bn2(self.conv4(x)))
        x = self.pool3(x)

        x = F.relu(self.conv5(x))
        x = self.pool4(x)

        x = self.flat(x)

        x = F.relu(self.fc1(x))
        x = self.drop1(x)
        
        x = F.relu(self.fc2(x))
        x = self.drop2(x)

        x = self.fc3(x)
        return x


# Training the model
Implement your training process below. Report the test-accuracy after every epoch for the training run of the final model.

Hint: before training your model make sure to reset the seed in the training cell, as otherwise the seed may have changed due to previous training runs in the notebook

Note: If you implement automatic hyperparameter tuning, split the train set into train and validation subsets for the objective function.

In [20]:
### HELPER FUNCTIONS ###

def train(model, dataloader, optimizer, loss_fn):
    """
    One epoch of model training.
    """
    model.train()

    running_loss = 0.0
    running_accuracy = 0.0
    for idx, data in enumerate(dataloader):
        images, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()
        outputs = model(images)
        correct = sum(labels == torch.argmax(outputs, 1)).item()
        running_accuracy += correct/batch_size

        loss = loss_fn(outputs, labels)
        running_loss += loss.item()
        loss.backward()
        optimizer.step()

        if idx % 500 == 499:
            avg_loss_across_batches = running_loss / 500
            avg_acc_across_batches = (running_accuracy / 500) * 100
            print('Batch {}, Loss: {:.3f}, Accuracy: {:.2f}%'.format(idx+1, avg_loss_across_batches, avg_acc_across_batches))

            running_loss = 0.0
            running_accuracy = 0.0


def validate(model, dataloader, loss_fn):
    """
    One validation test.
    """
    model.eval()

    running_loss = 0.0
    running_accuracy = 0.0
    for data in iter(dataloader):
        images, labels = data[0].to(device), data[1].to(device)

        with torch.no_grad():
            outputs = model(images)
            correct = sum(labels == torch.argmax(outputs, 1)).item()
            running_accuracy += correct/batch_size

            loss = loss_fn(outputs, labels)
            running_loss += loss.item()

    avg_loss_across_batches = running_loss / len(dataloader)
    avg_acc_across_batches = (running_accuracy / len(dataloader)) * 100
    
    print('Val Loss: {:.3f}, Val Accuracy: {:.2f}%'.format(avg_loss_across_batches, avg_acc_across_batches))
    return avg_loss_across_batches


def accuracy(model, dataloader):
    """
    One test accuracy test.
    """
    model.eval()

    running_accuracy = 0.0
    for data in iter(dataloader):
        images, labels = data[0].to(device), data[1].to(device)

        with torch.no_grad():
            outputs = model(images)
            correct = sum(labels == torch.argmax(outputs, 1)).item()
            running_accuracy += correct/batch_size

    avg_acc_across_batches = (running_accuracy / len(dataloader)) * 100
    print('Accuracy: {:.2f}%'.format(avg_acc_across_batches))
    return avg_acc_across_batches

In [ ]:
### TRAINING BLOCK ###

# Allow repeatability
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)

# Create model object and connect it to the corresponding processing
model = FoodCNN().to(device)

# Define loss and optimizer
loss_fn = nn.CrossEntropyLoss() 
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)

# Set up emotions for automation
regret = 10.0
grudge = 0
pride = 0.0

# Begin training, still includes developmental sections
for epoch in range(num_epochs):
    # Start epoch
    print(f"Epoch {epoch+1} begin: {dt.now()}.")
    start = time()

    # Train the model
    train(model, train_loader, optimizer, loss_fn)
    print('Completed training after {:.2f} seconds.'.format((time() - start)))
    check_point = time()

    # Validate the model
    val_loss = validate(model, val_loader, loss_fn)
    print('Completed validation after {:.2f} seconds.'.format((time() - check_point)))
    check_point = time()

    # Test the model
    acc = accuracy(model, test_loader)
    print('Completed testing after {:.2f} seconds.'.format((time() - check_point)))

    # Output an update
    print('Total epoch time: {:.2f} seconds.'.format((time() - start)))
    print('Epoch [{}/{}]'.format(epoch+1, num_epochs))
    check_point = time()

    # Update emotional state
    if pride < acc:
        print("Check-point!")
        torch.save(model.state_dict(), "./checkpoint.pth")
    grudge = 0 if val_loss <= regret else grudge + 1
    regret = val_loss
    if 2 < grudge:
        print("Overfitting!")
        torch.save(model.state_dict(), "./endpoint.pth")
        break

    print(time() - check_point)
    print("####################################################################################################")

Epoch 1 begin: 2025-05-03 23:37:41.458601.
Batch 500, Loss: 4.510, Accuracy: 1.20%
Batch 1000, Loss: 4.502, Accuracy: 1.82%
Batch 1500, Loss: 4.487, Accuracy: 2.43%
Batch 2000, Loss: 4.467, Accuracy: 2.45%
Batch 2500, Loss: 4.422, Accuracy: 2.81%
Completed training after 233.60 seconds.
Val Loss: 4.363, Val Accuracy: 3.47%
Completed validation after 18.19 seconds.
Accuracy: 3.50%
Completed testing after 46.85 seconds.
Total epoch time: 298.64 seconds.
Epoch [1/10]
####################################################################################################
Epoch 2 begin: 2025-05-03 23:42:40.099796.
Batch 500, Loss: 4.371, Accuracy: 3.49%
Batch 1000, Loss: 4.329, Accuracy: 3.90%
Batch 1500, Loss: 4.310, Accuracy: 4.46%
Batch 2000, Loss: 4.267, Accuracy: 4.89%
Batch 2500, Loss: 4.222, Accuracy: 5.33%
Completed training after 232.78 seconds.
Val Loss: 4.137, Val Accuracy: 7.04%
Completed validation after 20.13 seconds.
Accuracy: 7.00%
Completed testing after 47.75 seconds.
Total ep

In [ ]:
for epoch in range(epoch+1, epoch+11):
    # Start epoch
    print(f"Epoch {epoch+1} begin: {dt.now()}.")
    start = time()

    # Train the model
    train(model, train_loader, optimizer, loss_fn)
    print('Completed training after {:.2f} seconds.'.format((time() - start)))
    check_point = time()

    # Validate the model
    val_loss = validate(model, val_loader, loss_fn)
    print('Completed validation after {:.2f} seconds.'.format((time() - check_point)))
    check_point = time()

    # Test the model
    acc = accuracy(model, test_loader)
    print('Completed testing after {:.2f} seconds.'.format((time() - check_point)))

    # Output an update
    print('Total epoch time: {:.2f} seconds.'.format((time() - start)))
    print('Epoch [{}/{}]'.format(epoch+1, num_epochs))
    check_point = time()

    # Update emotional state
    if pride < acc:
        print("Check-point!")
        torch.save(model.state_dict(), "./checkpoint.pth")
    grudge = 0 if val_loss <= regret else grude + 1
    regret = val_loss
    if 2 < grude:
        print("Overfitting!")
        torch.save(model.state_dict(), "./endpoint.pth")
        break

    print(time() - check_point)
    print("####################################################################################################")

Epoch 43 begin: 2025-05-04 11:04:04.612512.
Batch 500, Loss: 1.665, Accuracy: 55.26%
Batch 1000, Loss: 1.674, Accuracy: 53.84%
Batch 1500, Loss: 1.670, Accuracy: 54.51%
Batch 2000, Loss: 1.745, Accuracy: 53.29%
Batch 2500, Loss: 1.707, Accuracy: 53.81%
Completed training after 234.54 seconds.
Val Loss: 2.489, Val Accuracy: 38.37%
Completed validation after 19.09 seconds.


In [ ]:
torch.save(model.state_dict(), "./pause.pth")

In [8]:
### MODEL SAVING ###
print("Overwrite? \"Yes\" / else: ")
user_input = input().lower()
if user_input == "yes" or user_input == 'y':
    PATH = './cnn.pth'
    torch.save(model.state_dict(), PATH)
    print("Save complete.")
else:
    PATH = './cnn_backup.pth'
    torch.save(model.state_dict(), PATH)
    print("Original unchanged, Backup overwritten.")

Overwrite? "Yes" / else: 
Save complete.


# Calculating model performance
Load the best version of your model ( which should be produced and saved by previous cells ), calculate and report the test accuracy.

In [ ]:
### MODEL COMPARISONS ###

# # Load the best model weights
# model2 = FoodCNN().to(device)
# model2.load_state_dict(torch.load("cnn.pth", weights_only=True))

# # Calculate and present final scores
# final_test_acc = accuracy(model, test_loader)
# print(f"Final Test Accuracy: {final_test_acc:.2f}%")


# Summary of hyperparameters
Report the hyperparameters ( learning rate etc ) that you used in your final model for reproducibility.

# Simulation of random user
Pick 10 random pictures of the test set to simulate a user uploading images and report which categories occur how often in these: 1pt

In [10]:
# Your code here
# Below an example showing the format of the code output

# Bonus point
Use an LLM (API) to generate a description of the food preference of a user based on 10 images that a potential user could provide. 
Please include an example of the output of your code, especially if you used an API other than the OpenAI API.

This should work well even with differing test images by setting different random seeds for the image selector.